In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import requests as rq
import tensorflow as tf

2025-02-08 23:39:17.974225: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-08 23:39:18.113848: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739079558.168900    4698 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739079558.184915    4698 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-08 23:39:18.321376: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Model Plan:
- Find a api for real time stock data
- Find a api for real time news
- Feature engineer features
- Build Multi-modal model for day trading.
- First branch of model will be Time Series analysis model that will take the real time stock data as a sequence
- Second branch of model will be a text classifier/sentiment analysis model that can take the real time news
- Combine the two branches to make the multi-modal model

# Agent Plan:
- Use same apis to get input data (This is how agent recieves input)
- Use LLM with function calling to helper function and to run the model (Thinking part)
- Have the agent output the result or make the trade itself using the api (The action/goal achieving part)


## Data Collection

In [54]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=5min&apikey=YWK0V0LYMQ4AD03I'

In [59]:
response = rq.get(url)
data = response.json()
print(response.content)
meta_data = data.pop('Meta Data')
meta_data

b'{\n    "Information": "Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits."\n}'


KeyError: 'Meta Data'

In [56]:
df = pd.DataFrame.from_dict(data)
df['Time Series (5min)'].iloc[0]

ValueError: If using all scalar values, you must pass an index

In [57]:
df['Open'] = df['Time Series (5min)'].apply(lambda x: x.get('1. open'))
df['High'] = df['Time Series (5min)'].apply(lambda x: x.get('2. high'))
df['Low'] = df['Time Series (5min)'].apply(lambda x: x.get('3. low'))
df['Close'] = df['Time Series (5min)'].apply(lambda x: x.get('4. close'))
df['Volume'] = df['Time Series (5min)'].apply(lambda x: x.get('5. volume'))
df['Ticker'] = meta_data.get('2. Symbol')

df.drop('Time Series (5min)', axis=1, inplace=True)

KeyError: 'Time Series (5min)'

In [58]:
df

""


In [42]:
list_of_tickers = ['AAPL', 'NVDA', 'MSFT', 'AMZN', 'META', 'GOOGL', 'TSLA']

In [53]:
### Build full dataset 
df = pd.DataFrame()
for ticker in list_of_tickers:
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval=5min&apikey=YWK0V0LYMQ4AD03I'
    response = rq.get(url)
    data = response.json()
    try:
        meta_data = data.pop('Meta Data')
        print(meta_data)
        meta_data
        temp_df = pd.DataFrame.from_dict(data)
        print(temp_df.head())
        temp_df['Open'] = temp_df['Time Series (5min)'].apply(lambda x: x.get('1. open'))
        temp_df['High'] = temp_df['Time Series (5min)'].apply(lambda x: x.get('2. high'))
        temp_df['Low'] = temp_df['Time Series (5min)'].apply(lambda x: x.get('3. low'))
        temp_df['Close'] = temp_df['Time Series (5min)'].apply(lambda x: x.get('4. close'))
        temp_df['Volume'] = temp_df['Time Series (5min)'].apply(lambda x: x.get('5. volume'))
        temp_df['Ticker'] = meta_data.get('2. Symbol')
        temp_df.drop('Time Series (5min)', axis=1, inplace=True)
    
        print(temp_df.head())
        pd.concat([df, temp_df], axis=0)
    except Exception as err:
        print(f"Error:{err} on ticker: {ticker}")
    


Error:'Meta Data' on ticker: AAPL
Error:'Meta Data' on ticker: NVDA
Error:'Meta Data' on ticker: MSFT
Error:'Meta Data' on ticker: AMZN
Error:'Meta Data' on ticker: META
Error:'Meta Data' on ticker: GOOGL
Error:'Meta Data' on ticker: TSLA


In [46]:
df.head()

""
